In [20]:
from fastai.vision import *
from fastai.datasets import *
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import torchvision as ptv
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from utils.blocks import *
from utils.clr import *

from collections import OrderedDict
from pathlib import Path

import matplotlib.pyplot as plt
from scipy import signal

[autoreload of fastai.torch_core failed: Traceback (most recent call last):
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 246, in check
    superreload(m, reload, self.old_objects)
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 369, in superreload
    module = reload(module)
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/importlib/__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 678, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/home/paperspace/fastai/fastai/torch_core.py", line 87, in <module>
    class Module(nn.Module, metac

[autoreload of fastai.vision.models.xception failed: Traceback (most recent call last):
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 246, in check
    superreload(m, reload, self.old_objects)
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 369, in superreload
    module = reload(module)
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/importlib/__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 678, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/home/paperspace/fastai/fastai/vision/models/xception.py", line 24, in <module>
    class

In [5]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import pandas as pd
# import numpy as np
# import torchvision as ptv
# from fastai.dataset import *
# from fastai.conv_learner import *
# from torch.utils.data import DataLoader
# from torchvision.datasets import MNIST
# from utils.blocks import *
# from utils.clr import *

# from collections import OrderedDict
# from pathlib import Path

# import matplotlib.pyplot as plt
# from scipy import signal

ModuleNotFoundError: No module named 'fastai.dataset'

In [6]:
use_cuda = torch.cuda.is_available(); print(use_cuda)
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True
CUDA_LAUNCH_BLOCKING=1
%matplotlib inline
%reload_ext autoreload
%autoreload 2

True


In [7]:
bs = 20
sz = 28

In [8]:
PATH = Path("../data/mnist")
train = MNIST(PATH, train=True, transform=ptv.transforms.ToTensor())
val = MNIST(PATH, train=False, transform=ptv.transforms.ToTensor())

trn_dl = DataLoader(train, batch_size=bs)
val_dl = DataLoader(val, batch_size=bs)

In [13]:
trn_dl.dataset.train_data[1].shape

torch.Size([28, 28])

In [6]:
# imgs = df[:int(len(df)*params["t_pc"])].drop("label", axis =1)
# imgs = imgs/255.0
# val_idxs = get_cv_idxs(len(imgs))
# ((x_val, x_train),) = split_by_idx(val_idxs, imgs)

In [28]:
# y_val = ys[ys.index.isin(x_val.index)]
# y_train = ys[ys.index.isin(x_train.index)]
# def conv_calc(w,q,p,s):
#     y = (w-q+2*p)/s+1
#     print(y)
#     return y

# q = conv_calc(28,4,1,2)
# q = conv_calc(q,4,1,2)
# ??untar_data
untar_data(URLs.MNIST,dest=PATH)

PosixPath('../data/mnist/mnist_png')

In [8]:
# x_val = x_val.values.reshape(-1,1,params["sz"],params["sz"])
# x_train = x_train.values.reshape(-1,1,params["sz"],params["sz"])

In [9]:
md = ImageClassifierData.from_arrays(PATH, (x_train, x_train), (x_val, x_val), bs=params["bs"])

In [14]:
class StdConv(nn.Module):
    def __init__(self, nin, nout, stri=2, kern=4, pad=1, drop=0.1, relu=True):
        super().__init__()
        self.conv = nn.Conv2d(nin, nout, kernel_size=kern, stride=stri, padding=pad,bias=False)
        self.bn = nn.BatchNorm2d(nout)
        self.drop = nn.Dropout(drop)
        self.relu = relu
    def forward(self, x):
        if self.relu:
            return self.drop(self.bn(F.relu(self.conv(x))))
        else:
            return self.drop(self.bn(self.conv(x)))
        
class StdUpsample(nn.Module):
    def __init__(self, nin, nout, pad=0):
        super().__init__()
        self.conv = nn.ConvTranspose2d(nin, nout, 2, stride=2,padding=pad)
        self.bn = nn.BatchNorm2d(nout)
        
    def forward(self, x): return self.bn(F.relu(self.conv(x)))
    
class StdLinear(nn.Module):
    def __init__(self, nin, nout, relu=True):
        super().__init__()
        self.lin = nn.Linear(nin, nout)
        self.relu = relu
    def forward(self, x):
        if self.relu: return F.relu(self.lin(x))
        return self.lin(x)

class VAE(nn.Module):
    def __init__(self, n_latent):
        super().__init__()
        self.n_latent = n_latent
        self.z_dim = 28//2**2
        self.h_dim = 64 * self.z_dim**2
        
        #encoder
        self.conv1 = StdConv(1,32)
        self.conv2 = StdConv(32,64)
        
        #q
        self.q_mean = StdLinear(self.h_dim, self.n_latent)
        self.q_logvar = StdLinear(self.h_dim, self.n_latent)
        self.q_project = StdLinear(n_latent, self.h_dim, False)
        
        #decoder
        self.upconv1 = StdUpsample(64, 32)
        self.upconv2 = StdUpsample(32,1)
    
    def encode(self, x):
        x = self.conv1(x)
        self.xc1 = x
        x = self.conv2(x)
        self.xc2 = x
        return x.view(x.size(0),-1)
    
    def z(self, flattened):
        self.mean, self.logvar = self.q_mean(flattened), self.q_logvar(flattened)
        std = self.logvar.mul(.5).exp_()
        eps = Variable(torch.randn(std.size())).cuda()
        return eps.mul(std).add_(self.mean)
    
    def decode(self, z, train=True):
        z = self.q_project(z).view(-1, 64, self.z_dim, self.z_dim)
        if train: torch.cat([z,self.xc2], dim=1)
        z = self.upconv1(z)
        if train: torch.cat([z,self.xc1], dim=1)
        z = self.upconv2(z)
        return z
    
    def forward(self, x):
        flattened = self.encode(x)
        z = self.z(flattened)
        decoded = self.decode(z)
        
        return (decoded, (self.mean, self.logvar))

In [15]:
class StableBCELoss(nn.modules.Module):
    def __init__(self):
         super(StableBCELoss, self).__init__()
    def forward(self, input, target):
        neg_abs = - input.abs()
        loss = input.clamp(min=0) - input * target + (1 + neg_abs.exp()).log()
        return loss.mean()
    
def loss(output, x):
    return nn.MSELoss(size_average=False)(output, x)
    
def kl_loss(decoded, xtras, raw_loss):
    if isinstance(xtras, list): xtras = xtras[0]
    mean, logvar = xtras
    kl = -0.5 * torch.sum(1 + logvar - mean.pow(2) - logvar.exp())
    return kl*.7 + raw_loss

def vae_loss(output, x):
    decoded, xtras = output
    recon_loss = nn.MSELoss()(decoded, x)
    loss = kl_loss(decoded, xtras, recon_loss)
    return loss, recon_loss, loss - recon_loss

In [16]:
net = VAE(16)
# learn = ConvLearner.from_model_data(net, md)
learn = ConvLearner.from_model_data(net,trn_dl)

NameError: name 'ConvLearner' is not defined

In [ ]:
learn.load("U-CVAE4")

In [ ]:
x,y = next(iter(md.val_dl))
x = V(x)
p = learn.model(x)
vae_loss(p, x)

In [ ]:
learn.model.cuda()
learn.opt_fn = optim.Adam
learn.crit = loss
learn.reg_fn = kl_loss

learn.lr_find()
learn.sched.plot()

In [ ]:
lr = 2e-5
learn.fit(lr, 1 ,cycle_len=20, use_clr=(20,10),metrics=[])

In [ ]:
# learn.save("U-CVAE4")

In [ ]:
x,y = next(iter(md.val_dl))
x,y = V(x), V(y)
# plt.imshow(x[3][0,:,:])
im,x = learn.model(x)
plt.imshow(im.data[2].cpu().numpy().reshape(28,28)*255.)

In [ ]:
g = iter(md.val_dl)

In [ ]:
def generate(bs=params["bs"]):
    vx = torch.FloatTensor([np.random.normal(0, 1, params["sz"]**2) for _ in range(bs)]).cuda()
    vx = V(vx).view(-1,1,params["sz"],params["sz"])
    return learn.model(vx)

def display_recon(g=iter(md.val_dl),bs=params["bs"]//2):
    x, _ = next(g)
    vx = V(x[:bs])
    z_im, _ = learn.model(vx)
    z_im = z_im.data.cpu().numpy()
    fig, axes = plt.subplots(2, bs, figsize=(10,3))
    for ax, im in enumerate(x[:bs]):
        axes[0,ax].imshow(im[0,:,:])
        axes[1,ax].imshow(z_im[ax].reshape(28,28)*255.)
    plt.tight_layout()
    
def display_gen(bs=params["bs"]):
    ims, xtras = generate(bs)
    ims = ims.data.cpu().numpy()
    ims = ims.reshape(-1,28,28)*255.
    plt.figure(figsize=(10,3))
    for i, im in enumerate(ims):
        plt.subplot(2,bs/2,i+1)
        plt.imshow(im)
    plt.tight_layout()

In [ ]:
display_recon(g); display_gen()

In [ ]:
display_recon(g); display_gen()

In [ ]:
# x, _ = next(g)
# vx = V(x[:params["bs"]])
# e = learn.model.encode(vx)
# z = learn.model.z(e)
d2= learn.model.decode(z)
d = learn.model.decode(z, False)
z_im2 = d.data.cpu().numpy()
z_im = d.data.cpu().numpy()
fig, axes = plt.subplots(3, params["bs"], figsize=(14,3))
for ax, im in enumerate(x[:params["bs"]]):
    axes[0,ax].imshow(im[0,:,:])
    axes[1,ax].imshow(z_im[ax].reshape(28,28)*255.)
    axes[2,ax].imshow(z_im2[ax].reshape(28,28)*255.)
plt.tight_layout()